In [14]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from ezc3d import c3d
from scipy.signal import butter, filtfilt

In [15]:
def lowpass_filter(data, cutoff=5, fs=100, order=4):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data, axis=0)
    return y

def extract_gait_cycle(c, gait_duration=3):

    times = c["parameters"]["EVENT"]["TIMES"]['value']
    contexts = c["parameters"]["EVENT"]["CONTEXTS"]['value']
    labels = c["parameters"]["EVENT"]["LABELS"]['value']
    gait_points = []

    for i in range(len(labels)):
        if (labels[i] == "Foot Strike1") and (contexts[i] == "Right"):
            gait_start = int(times[1,i]*100)
        if (labels[i] == "Foot Strike2") and (contexts[i] == "Right"):
            gait_end = int(times[1,i]*100)

    trial_data = c['data']['points'][:,:,gait_start:gait_end]
    labels = c['parameters']['POINT']['LABELS']['value']

    #[15,4,16,5,20,9]
    r_ftc = labels.index("R_FTC") if "R_FTC" in labels else RuntimeError("R_FTC not found")
    l_ftc = labels.index("L_FTC") if "L_FTC" in labels else RuntimeError("L_FTC not found")
    r_fle = labels.index("R_FLE") if "R_FLE" in labels else RuntimeError("R_FLE not found")
    l_fle = labels.index("L_FLE") if "L_FLE" in labels else RuntimeError("L_FLE not found")
    r_fal = labels.index("R_FCC") if "R_FCC" in labels else RuntimeError("R_FCC not found")
    l_fal = labels.index("L_FCC") if "L_FCC" in labels else RuntimeError("L_FCC not found")
    sxs = labels.index("SXS") if "SXS" in labels else RuntimeError("SXS not found")

    r_leglength_ind = c['parameters']['SUBJECT']['LABELS']['value'].index("R_legLength") if "R_legLength" in c['parameters']['SUBJECT']['LABELS']['value'] else RuntimeError("R_LEGLENGTH not found")
    l_leglength_ind = c['parameters']['SUBJECT']['LABELS']['value'].index("L_legLength") if "L_legLength" in c['parameters']['SUBJECT']['LABELS']['value'] else RuntimeError("L_LEGLENGTH not found")
    weight = c['parameters']['SUBJECT']['LABELS']['value'].index("weight") if "weight" in c['parameters']['SUBJECT']['LABELS']['value'] else RuntimeError("Weight not found")
    height = c['parameters']['SUBJECT']['LABELS']['value'].index("height") if "height" in c['parameters']['SUBJECT']['LABELS']['value'] else RuntimeError("Height not found")
    r_leglength = c['parameters']['SUBJECT']['VALUES']['value'][r_leglength_ind]
    l_leglength = c['parameters']['SUBJECT']['VALUES']['value'][l_leglength_ind]
    weight = c['parameters']['SUBJECT']['VALUES']['value'][weight]
    height = c['parameters']['SUBJECT']['VALUES']['value'][height]

    muscle_index = [r_ftc,l_ftc,r_fle,l_fle,r_fal,l_fal]
    joint_data = trial_data[:,muscle_index,:]
    joint_states = np.zeros((joint_data.shape[2],4))

    data_len = joint_data.shape[2]


    first_pos = trial_data[0,sxs,0]
    last_pos = trial_data[0,sxs,-1]

    if last_pos > first_pos:
        forward_walk = True
    else:
        forward_walk = False

    speed_muscle = trial_data[:,sxs,:]
    speed = np.abs(speed_muscle[0,-1]-speed_muscle[0,0])/(len(speed_muscle[0,:]))/10

    #data_point[:,-1] = speed

    if forward_walk:
        for i in range(data_len):
            joint_states[i,0] = np.arctan2((joint_data[0,2,i] - joint_data[0,0,i]) , (-joint_data[2,2,i] + joint_data[2,0,i])) 
            joint_states[i,1] = np.arctan2((joint_data[0,4,i] - joint_data[0,2,i]) , (-joint_data[2,4,i] + joint_data[2,2,i])) - joint_states[i,0]
            joint_states[i,2] = np.arctan2((joint_data[0,3,i] - joint_data[0,1,i]) , (-joint_data[2,3,i] + joint_data[2,1,i]))
            joint_states[i,3] = np.arctan2((joint_data[0,5,i] - joint_data[0,3,i]) , (-joint_data[2,5,i] + joint_data[2,3,i])) - joint_states[i,2]
            # joint_states[i,4] = speed
            # joint_states[i,5] = r_leglength
            # joint_states[i,6] = l_leglength
    else:
        for i in range(data_len):
            joint_states[i,0] = np.arctan2((-joint_data[0,2,i] + joint_data[0,0,i]) , (-joint_data[2,2,i] + joint_data[2,0,i])) 
            joint_states[i,1] = np.arctan2((-joint_data[0,4,i] + joint_data[0,2,i]) , (-joint_data[2,4,i] + joint_data[2,2,i])) - joint_states[i,0]
            joint_states[i,2] = np.arctan2((-joint_data[0,3,i] + joint_data[0,1,i]) , (-joint_data[2,3,i] + joint_data[2,1,i]))
            joint_states[i,3] = np.arctan2((-joint_data[0,5,i] + joint_data[0,3,i]) , (-joint_data[2,5,i] + joint_data[2,3,i])) - joint_states[i,2]
            # joint_states[i,4] = speed
            # joint_states[i,5] = r_leglength
            # joint_states[i,6] = l_leglength
    total_gait_sample = 420
    joint_states = lowpass_filter(joint_states, cutoff=10, fs=100, order=4)

    if joint_states.shape[0] > total_gait_sample:
        print(f'gait cycle: {joint_states.shape[1]} longer than {gait_duration} seconds')
        
    while joint_states.shape[0] < total_gait_sample:
        joint_states = np.concatenate((joint_states, joint_states),axis=0)  # Stack horizontally

    joint_states = joint_states[:total_gait_sample,:]
    padded_joint_states = np.zeros((512, 4))
    padded_joint_states[46:46+joint_states.shape[0],:] = joint_states
    
    joint_states_rfft = np.fft.rfft(padded_joint_states, axis=0, n=512)
    real_fft = np.real(joint_states_rfft)
    imag_fft = np.imag(joint_states_rfft)
    joint_states_rfft = np.stack((real_fft, imag_fft), axis=2)
    freq_values = np.fft.rfftfreq(512, 1/100)
    encoder_vec = np.empty((3))   # init_pos + speed + r_leglength + l_leglength + ramp_angle = 0
    # encoder_vec[0:4] = joint_states[:,0]
    encoder_vec[0] = speed/3
    encoder_vec[1] = r_leglength
    encoder_vec[2] = l_leglength
    # encoder_vec[3] = weight / 100  # 100 is the maximum weight in the dataset
    # encoder_vec[4] = height / 2    # 1.91 m is the maximum height in the dataset

    encoder_vec = encoder_vec[np.newaxis, :]
    joint_states_rfft = joint_states_rfft[np.newaxis, :, :] #179 is the maximum value in the dataset
    return joint_states_rfft, freq_values, encoder_vec, joint_states[10:-10,:]

In [16]:
# def create_ifft_plots(joint_states_rfft,joint_states,frequencies,path,crop_ind=30):
#     frequency = frequencies[crop_ind]
#     joint_states_rfft = 180*np.squeeze(joint_states_rfft)
#     cropped = joint_states_rfft.copy()
#     cropped[crop_ind:,:] = 0
#     ifft = np.fft.irfft(cropped, axis=0)
#     cropped_ifft = ifft[56:-56,:]
#     fig, axs = plt.subplots(2, 2, figsize=(10, 8))
#     axs[0, 0].plot(joint_states[:,0])
#     axs[0, 0].plot(cropped_ifft[:,0])
#     axs[0, 0].legend(['Original', 'Reconstructed'])
#     axs[0, 0].set_title('Right Shank')
#     axs[0, 1].plot(joint_states[:,1])
#     axs[0, 1].plot(cropped_ifft[:,1])
#     axs[0, 1].set_title('Right Thigh')
#     axs[0, 1].legend(['Original', 'Reconstructed'])
#     axs[1, 0].plot(joint_states[:,2])
#     axs[1, 0].plot(cropped_ifft[:,2])
#     axs[1, 0].set_title('Left Shank')
#     axs[1, 0].legend(['Original', 'Reconstructed'])
#     axs[1, 1].plot(joint_states[:,3])
#     axs[1, 1].plot(cropped_ifft[:,3])
#     axs[1, 1].set_title('Left Thigh')
#     axs[1, 1].legend(['Original', 'Reconstructed'])
#     plt.savefig(path+'.png')
#     plt.close() 

In [ ]:
import os
gait_duration = 2
crop_ind = 25
folders = [d for d in os.listdir(os.path.join(os.getcwd(),'dataset')) if os.path.isdir(os.path.join(os.path.join(os.getcwd(),'dataset'), d))]
k=0
if os.path.exists(f"gait reference fft_{crop_ind}_plots") == False:
    os.mkdir(f"gait reference fft_{crop_ind}_plots")

for folder in folders:
    folder_len = len(folder)
    files = [f for f in os.listdir(os.path.join(os.getcwd(),'dataset',folder)) if f.endswith('.c3d')]
    
    print(f"{k}/{len(folders)} is processed current folder is {folder}") 
    i = 0
    for file in files:
        if 'ST' not in file:
            c = c3d(os.path.join('dataset',folder,file))
            joint_states_rfft, freq_values, encoder_vec, joint_states = extract_gait_cycle(c,gait_duration=gait_duration)
            # if np.max(np.abs(joint_states)) > 1:
            #     print(f'joint_states max value is greater than 1 in {folder}_{file[:-4]}')
            #     break
            if i == 0:
                folder_output_state = joint_states_rfft
                folder_input_vector = encoder_vec
            else:
                folder_output_state = np.vstack((folder_output_state,joint_states_rfft))
                folder_input_vector = np.vstack((folder_input_vector,encoder_vec))
            i+=1
    print('folder_output_state_shape: ',folder_output_state.shape)
    print('folder input vector shape: ',folder_input_vector.shape)
    print('-----------------------------------')
    if k == 0:
        total_output_state = folder_output_state
        total_input_vector = folder_input_vector
    else:
        total_output_state = np.vstack((total_output_state,folder_output_state))
        total_input_vector = np.vstack((total_input_vector,folder_input_vector))
    k+=1
#     np.save(f"ref_gait_library_duration{gait_duration}/{folder}_output_state.npy",folder_output_state)
#     np.save(f"ref_gait_library_duration{gait_duration}/{folder}_input_vector.npy",folder_input_vector)

if os.path.exists(f"gait reference fft_{crop_ind}_{freq_values[crop_ind]:.2f}") == False:
    os.mkdir(f"gait reference fft_{crop_ind}_{freq_values[crop_ind]:.2f}")

np.save(f"gait reference fft_{crop_ind}_{freq_values[crop_ind]:.2f}/output_fft_constants.npy",total_output_state)
np.save(f"gait reference fft_{crop_ind}_{freq_values[crop_ind]:.2f}/input_vector.npy",total_input_vector)

print('total output state ',total_output_state.shape)
print('total input vector ',total_input_vector.shape)

In [ ]:
# NORMALIZATION OF THE DATA IS IMPLEMENTED
import numpy as np
real_rthigh = np.max(np.abs(total_output_state[:,:,0,0]))
imag_rthigh = np.max(np.abs(total_output_state[:,:,0,1]))
real_rshank = np.max(np.abs(total_output_state[:,:,1,0]))
imag_rshank = np.max(np.abs(total_output_state[:,:,1,1]))
real_lthigh = np.max(np.abs(total_output_state[:,:,2,0]))
imag_lthigh = np.max(np.abs(total_output_state[:,:,2,1]))
real_lshank = np.max(np.abs(total_output_state[:,:,3,0]))
imag_lshank = np.max(np.abs(total_output_state[:,:,3,1]))

print(real_rthigh,imag_rthigh,real_rshank,imag_rshank,real_lthigh,imag_lthigh,real_lshank,imag_lshank) 

normalizationconst = np.array([real_rthigh,imag_rthigh,real_rshank,imag_rshank,real_lthigh,imag_lthigh,real_lshank,imag_lshank])

total_output_state[:,:,0,0] = total_output_state[:,:,0,0] / real_rthigh
total_output_state[:,:,1,0] = total_output_state[:,:,1,0] / real_rshank
total_output_state[:,:,2,0] = total_output_state[:,:,2,0] / real_lthigh
total_output_state[:,:,3,0] = total_output_state[:,:,3,0] / real_lshank

total_output_state[:,:,0,1] = total_output_state[:,:,0,1] / imag_rthigh
total_output_state[:,:,1,1] = total_output_state[:,:,1,1] / imag_rshank
total_output_state[:,:,2,1] = total_output_state[:,:,2,1] / imag_lthigh
total_output_state[:,:,3,1] = total_output_state[:,:,3,1] / imag_lshank

np.save(f"gait reference fft_{crop_ind}_{freq_values[crop_ind]:.2f}/normalized_output_fft_constants.npy",total_output_state)
np.save(f"gait reference fft_{crop_ind}_{freq_values[crop_ind]:.2f}/normalized_input_vector.npy",total_input_vector)

In [19]:
np.save(f"gait reference fft_{crop_ind}_{freq_values[crop_ind]:.2f}/normalization_constants.npy",normalizationconst)

**Training Part**

In [ ]:
import scipy
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

inputs = np.load(rf"gait reference fft_25_4.88\normalized_input_vector.npy")
outputs = np.load(rf"gait reference fft_25_4.88\normalized_output_fft_constants.npy")

#train test split using sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.01, random_state=23)

y_train = y_train[:,:25,:]
y_train  = y_train.transpose(0,3,1,2)
y_test = y_test[:,:25,:]
y_test = y_test.transpose(0,3,1,2)
#convert to tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

# create dataloader
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

print('train data shape: ',X_train.shape)
print('test data shape: ',X_test.shape)
print('train output shape: ',y_train.shape)
print('test output shape: ',y_test.shape)

In [30]:
def denormalize(pred,gt,normalizationconst):

    pred[0,:,0] = pred[0,:,0] * normalizationconst[0]
    pred[0,:,1] = pred[0,:,1] * normalizationconst[1]
    pred[0,:,2] = pred[0,:,2] * normalizationconst[2]
    pred[0,:,3] = pred[0,:,3] * normalizationconst[3]
    pred[1,:,0] = pred[1,:,0] * normalizationconst[4]
    pred[1,:,1] = pred[1,:,1] * normalizationconst[5]
    pred[1,:,2] = pred[1,:,2] * normalizationconst[6]
    pred[1,:,3] = pred[1,:,3] * normalizationconst[7]

    gt[0,:,0] = gt[0,:,0] * normalizationconst[0]
    gt[0,:,1] = gt[0,:,1] * normalizationconst[1]
    gt[0,:,2] = gt[0,:,2] * normalizationconst[2]
    gt[0,:,3] = gt[0,:,3] * normalizationconst[3]
    gt[1,:,0] = gt[1,:,0] * normalizationconst[4]
    gt[1,:,1] = gt[1,:,1] * normalizationconst[5]
    gt[1,:,2] = gt[1,:,2] * normalizationconst[6]
    gt[1,:,3] = gt[1,:,3] * normalizationconst[7]
    
    return pred,gt
    

def pred_ifft(predictions,ground_truth,speed,normalizationconst):

    real_pred = predictions[0,:,:]
    imag_pred = predictions[0,:,:]
    predictions = real_pred + 1j*imag_pred
    real_gt = ground_truth[0,:,:]
    imag_gt = ground_truth[1,:,:]
    ground_truth = real_gt + 1j*imag_gt
    
    padded_pred = np.zeros((257,4),dtype=complex)
    padded_pred[:25,:] = predictions
    padded_gt = np.zeros((257,4),dtype=complex)
    padded_gt[:25,:] = ground_truth

    padded_time = np.fft.irfft(padded_pred, axis=0)
    pred_time = padded_time[56:-56,:]
    gt_time = np.fft.irfft(padded_gt, axis=0)
    gt_time = gt_time[56:-56,:]
    #plot 2*2 subplots
    plt.figure(figsize=(10,8))
    plt.subplot(2,2,1)
    plt.plot(pred_time[:,0])
    plt.plot(gt_time[:,0])
    plt.title('Right Shank')
    plt.legend(['Predicted','Ground Truth'])
    plt.subplot(2,2,2)
    plt.plot(pred_time[:,1])
    plt.plot(gt_time[:,1])
    plt.title('Right Thigh')
    plt.legend(['Predicted','Ground Truth'])
    plt.subplot(2,2,3)
    plt.plot(pred_time[:,2])
    plt.plot(gt_time[:,2])
    plt.title('Left Shank')
    plt.legend(['Predicted','Ground Truth'])
    plt.subplot(2,2,4)
    plt.plot(pred_time[:,3])
    plt.plot(gt_time[:,3])
    plt.title('Left Thigh')
    plt.legend(['Predicted','Ground Truth'])
    plt.savefig(f"predict_plots_25hs512_fft/{speed:.1f}ms.png")
    plt.close()
    print('plots saved')
    return pred_time,gt_time

In [22]:
import torch
import torch.nn as nn

class WeightedMSELoss(nn.Module):
    def __init__(self, weights):
        super(WeightedMSELoss, self).__init__()
        self.weights = weights

    def forward(self, predictions, targets):
        # Element-wise weighted MSE
        loss = self.weights * (predictions - targets) ** 2
        return loss.mean()

# Example weights: first 10 have weight 10, the rest have weight 1

class large_penalized_mse(nn.Module):
    def __init__(self):
        super(large_penalized_mse, self).__init__()
    """
    Custom MSE loss function that penalizes the first 100 outputs more
    if their absolute values exceed the target values.
    
    Args:
        predictions: Tensor of shape (batch_size, 200)
        targets: Tensor of shape (batch_size, 200)
        weight_factor: Penalty factor for the first 100 outputs when they exceed the target.
    """
    def forward(self, predictions, targets):
        weight_factor=2.0
        # Compute standard MSE for all outputs
        mse_loss = (predictions - targets) ** 2
        
        # Separate the first 100 and last 100 outputs
        first_100_preds = predictions[:, :100]
        first_100_targets = targets[:, :100]
        
        # Condition for penalizing: predictions > targets (element-wise)
        penalty_mask = (first_100_preds > first_100_targets).float()
        
        # Apply the weight factor to the first 100 outputs where the condition is met
        weighted_first_100_loss = mse_loss[:, :100] * (1 + penalty_mask * (weight_factor - 1))
        
        # Combine the weighted first 100 loss and the regular last 100 loss
        combined_loss = torch.cat([weighted_first_100_loss, mse_loss[:, 100:]], dim=1)
        
        # Return the mean loss
        return combined_loss.mean()


In [ ]:
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F

class SimpleFCNN(nn.Module):
    def __init__(self, input_size=3, output_size=160, hidden_size=128):
        super(SimpleFCNN, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size)  # Input to hidden layer
        self.fc2 = nn.Linear(hidden_size, hidden_size) # Hidden to output layer
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()                          # Activation function

    def forward(self, x):
        # Apply layers with activation function
        x = self.fc1(x)         # Hidden layer
        x = nn.Dropout(0.5)(x)
        x = self.relu(x)        # Activation function
        x = self.fc2(x)         # Hidden layer
        x = nn.Dropout(0.5)(x)
        x = self.relu(x)        # Activation function
        x = self.fc3(x)         # Output layer
        return x

# Instantiate the model
torch.manual_seed(23)
input_size = 3
output_size = 200
hidden_size = 512
model = SimpleFCNN(input_size=input_size, output_size=output_size,hidden_size=hidden_size)
# Print the model architecture
print(model)

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

batch_size = 64

# Generate synthetic data for demonstration
weights = torch.tensor([1.0] * 200)
# weights[:40] = 3
# weights[100:40] = 3
# loss_fn = WeightedMSELoss(weights)
# loss_fn = nn.MSELoss()
loss_fn = large_penalized_mse()
# Create a dataset and DataLoader
dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
losses = []
# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in dataloader:
        targets = targets.reshape(-1,200)

        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        # Compute loss
        loss = loss_fn(outputs, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    losses.append(running_loss)
    
    # Print epoch loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")
plt.plot(losses)
print("Training complete.")

torch.save(model.state_dict(), f"model_hs{hidden_size}_lpmse_bs{batch_size}_epoch{num_epochs}_fft.pth")

In [31]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def animate_biped(angles,filename):
    """
    Animate a 2D bipedal robot given an n*4 array of angles.
    
    Parameters:
    -----------
    angles : np.ndarray
        A numpy array of shape (n,4), where n is the number of time steps.
        Each row: [right_shank_angle, right_thigh_angle, left_shank_angle, left_thigh_angle]
    """
    num_frames = angles.shape[0]
    seg_length = 1.0

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.grid(True)

    # Lines
    torso_line, = ax.plot([], [], 'k-', lw=2)
    right_thigh_line, = ax.plot([], [], 'b-', lw=2)
    right_shank_line, = ax.plot([], [], 'r-', lw=2)
    left_thigh_line, = ax.plot([], [], 'g-', lw=2)
    left_shank_line, = ax.plot([], [], 'm-', lw=2)

    # Scatter points for joints
    hip_marker = ax.scatter([], [], c='black', s=50)
    right_knee_marker = ax.scatter([], [], c='blue', s=50)
    right_foot_marker = ax.scatter([], [], c='red', s=50)
    left_knee_marker = ax.scatter([], [], c='green', s=50)
    left_foot_marker = ax.scatter([], [], c='magenta', s=50)

    # Initialization function for FuncAnimation
    def init():
        torso_line.set_data([], [])
        right_thigh_line.set_data([], [])
        right_shank_line.set_data([], [])
        left_thigh_line.set_data([], [])
        left_shank_line.set_data([], [])

        # Set empty offsets as a (0,2) shaped array
        hip_marker.set_offsets(np.empty((0, 2)))
        right_knee_marker.set_offsets(np.empty((0, 2)))
        right_foot_marker.set_offsets(np.empty((0, 2)))
        left_knee_marker.set_offsets(np.empty((0, 2)))
        left_foot_marker.set_offsets(np.empty((0, 2)))

        return (torso_line, right_thigh_line, right_shank_line, 
                left_thigh_line, left_shank_line, hip_marker, 
                right_knee_marker, right_foot_marker, 
                left_knee_marker, left_foot_marker)

    def update(frame):
        ax.set_title(f"Time Step: {frame + 1}")
        r_thigh_angle, r_shank_angle, l_thigh_angle, l_shank_angle = angles[frame]

        # Hip at origin
        hip_pos = np.array([0.0, 0.0])
        # Simple torso line upwards
        torso_top = np.array([0.0, 1.0])
        torso_line.set_data([torso_top[0], hip_pos[0]], [torso_top[1], hip_pos[1]])

        # Right leg
        right_knee = hip_pos + seg_length * np.array([np.sin(r_thigh_angle), -np.cos(r_thigh_angle)])
        right_foot = right_knee + seg_length * np.array([np.sin(r_thigh_angle + r_shank_angle), -np.cos(r_thigh_angle + r_shank_angle)])
        right_thigh_line.set_data([hip_pos[0], right_knee[0]], [hip_pos[1], right_knee[1]])
        right_shank_line.set_data([right_knee[0], right_foot[0]], [right_knee[1], right_foot[1]])

        # Left leg
        left_knee = hip_pos + seg_length * np.array([np.sin(l_thigh_angle), -np.cos(l_thigh_angle)])
        left_foot = left_knee + seg_length * np.array([np.sin(l_thigh_angle + l_shank_angle), -np.cos(l_thigh_angle + l_shank_angle)])
        left_thigh_line.set_data([hip_pos[0], left_knee[0]], [hip_pos[1], left_knee[1]])
        left_shank_line.set_data([left_knee[0], left_foot[0]], [left_knee[1], left_foot[1]])

        # Update markers with a 2D array shape
        hip_marker.set_offsets([hip_pos])
        right_knee_marker.set_offsets([right_knee])
        right_foot_marker.set_offsets([right_foot])
        left_knee_marker.set_offsets([left_knee])
        left_foot_marker.set_offsets([left_foot])

        return (torso_line, right_thigh_line, right_shank_line, 
                left_thigh_line, left_shank_line, hip_marker, 
                right_knee_marker, right_foot_marker, 
                left_knee_marker, left_foot_marker)

    ani = FuncAnimation(fig, update, frames=num_frames, init_func=init, interval=1, blit=False)
    ani.save(filename, fps=100)

In [ ]:
# Create a test dataset and DataLoader
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)  # Batch size of 5
norm_consts = np.load(rf"gait reference fft_25_4.88\normalization_constants.npy")

# Testing loop
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
test_loss = 0.0

k = 0
with torch.no_grad():  # No need to compute gradients during testing
    for inputs, targets in test_dataloader:
        speed = inputs[0,0].item()*3


        targets = targets.reshape(-1,200)
        # Forward pass
        outputs = model(inputs)
        
        # Compute loss
        loss = loss_fn(outputs, targets)
        test_loss += loss.item()
        ground_truth = targets.reshape(-1,2,25,4)
        predictions = outputs.reshape(-1,2,25,4)

        predictions = predictions.detach().numpy()
        predictions = predictions[0]
        ground_truth = ground_truth.detach().numpy()
        ground_truth = ground_truth[0]

        predictions, ground_truth = denormalize(predictions,ground_truth,norm_consts)
        pred_time,gt_time = pred_ifft(predictions,ground_truth,speed,norm_consts)
        animate_biped(pred_time,f"predict_plots_25hs512_fft/{speed:.1f}ms.gif")
        animate_biped(gt_time,f"predict_plots_25hs512_fft/{speed:.1f}ms.gif")
        k+=1
# Calculate average loss and accuracy
test_loss /= len(test_dataloader)  # Average test loss


# Print test results
print(f"Test Loss: {test_loss:.4f}")
plt.plot(targets[0].numpy())
plt.plot(outputs[0].numpy())
plt.legend(['Ground Truth', 'Predictions'])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def ifft(predictions):
    padded_pred = np.zeros((257,4),dtype=complex)
    pred_real = predictions[:,:,0]
    pred_imag = predictions[:,:,1]
    predictions = pred_real + 1j*pred_imag
    padded_pred[:20,:] = predictions*180
    padded_time = np.fft.irfft(padded_pred, axis=0)
    pred_time = padded_time[56:-56,:]*90
    return pred_time


def test_ifft(predictions):
    pred_real = predictions[:,:,0]
    pred_imag = predictions[:,:,1]
    predictions = pred_real + 1j*pred_imag
    time_signal = np.fft.irfft(predictions, axis=0)
    pred_time = time_signal[56:-56,:]*90
    return pred_time

# model.eval()
# output = model(X_test[0].reshape(1,5))
# target = y_test[0]
# print(target.shape)
# predictions = outputs.reshape(-1,20,4)
# predictions = predictions.detach().numpy()

# animate_biped(ifft_pred*(np.pi/2))
a = np.load(r"gait reference fft_20_3.91\normalized_output_fft_constants.npy")
a = a[12,:,:,:]
b = test_ifft(a)
animate_biped(b)
print(a.shape)
# animate_biped(a)